#Libraries and Data

In [1]:
#import libraries
import numpy as np
import pandas as pd
import itertools
from prophet import Prophet
import yfinance
import pickle
import xgboost as xgb
from datetime import datetime

from dataProcessing import DataProcessing
# Remove Future Warnings
import warnings
warnings.simplefilter(action='ignore', category=FutureWarning)




from sklearn.model_selection import train_test_split

# Machine Learning
from xgboost import XGBClassifier
from sklearn.model_selection import RandomizedSearchCV, cross_val_score
from sklearn.model_selection import RepeatedStratifiedKFold

# Evaluation
from sklearn.metrics import precision_score

# Reporting
import matplotlib.pyplot as plt

/Users/vipin/opt/anaconda3/envs/my_own/lib/python3.9/site-packages/tqdm/auto.py:21: TqdmWarning: IProgress not found. Please update jupyter and ipywidgets. See https://ipywidgets.readthedocs.io/en/stable/user_install.html
  from .autonotebook import tqdm as notebook_tqdm
Importing plotly failed. Interactive plots will not work.


In [2]:
def load_and_setup_data(sybmol,input_data):
    df = pd.read_csv("../stock_historical_data/{}.csv".format(sybmol))
    df.set_index("Date", inplace=True)
    new_data = pd.DataFrame(input_data)
    new_data.set_index("Date", inplace=True)
    concatenated_df = pd.concat([df,new_data])
    return concatenated_df

def fetch_stock_data(tickers):
    data = yfinance.download(tickers,period='1d')
    print(data)
    return data
    

In [17]:
symbols = ["TATAMOTORS.NS","IOC.NS"]
  # Add the tickers you want to fetch data for
loaded_models = {}
final_result = {}
stock_data = fetch_stock_data(symbols)
print(stock_data)
finaldata = {}
stock_name = "TATAMOTORS"
for symbol in symbols:
    print()
    input_data = {
                    # "Open" : stock_data.head(1).Open[ticker].values[0],
                    "Open" : [stock_data.head(1).Open[symbol].values[0]],
                    "Close" : [stock_data.head(1).High[symbol].values[0]],
                    "High" : [stock_data.head(1).High[symbol].values[0]],
                    "Low" : [stock_data.head(1).Low[symbol].values[0]], 
                    "Volume" : [stock_data.head(1).Volume[symbol].values[0]],
                    "Date":[np.datetime_as_string(stock_data.index, unit='D')[0]]
    }
    print(input_data)
    data = load_and_setup_data(symbol,input_data)
    data.to_csv(f"../stock_historical_data/{symbol}.csv")
    data = DataProcessing(symbol,data)
    # Specify Target
    data.df.loc[data.df["Range"].shift(-1) > data.df["Avg_Range"], "TARGET"] = 1
    data.df.loc[data.df["Range"].shift(-1) <= data.df["Avg_Range"], "TARGET"] = 0
    loaded_model = {}
    feature_item = {}
    with open('../TrainedModel/xg/{}_model_2.pkl'.format(symbol), 'rb') as f:
        loaded_model = pickle.load(f)
    with open('../TrainedModel/xg/{}_features.txt'.format(symbol), 'rb') as f:
        feature_item = pickle.load(f)
    print(feature_item)
    #final training
    loaded_models[symbol] = loaded_model
    print("features ",feature_item)
    df = data.df[feature_item]
    finaldata[symbol]= df 
    train_yhat = loaded_model.predict(df[-1:])
    train_yhat_proba = loaded_model.predict_proba(df[-1:])
    final_result[symbol] = [train_yhat_proba,train_yhat]
    



[*********************100%***********************]  2 of 2 completed
           Adj Close                Close                     High  \
              IOC.NS TATAMOTORS.NS IOC.NS TATAMOTORS.NS     IOC.NS   
Date                                                                 
2023-07-05      95.5        588.25   95.5        588.25  95.699997   

                                Low                     Open                \
           TATAMOTORS.NS     IOC.NS TATAMOTORS.NS     IOC.NS TATAMOTORS.NS   
Date                                                                         
2023-07-05         593.0  93.400002    584.950012  95.150002         592.0   

              Volume                
              IOC.NS TATAMOTORS.NS  
Date                                
2023-07-05  14273978       8715228  
           Adj Close                Close                     High  \
              IOC.NS TATAMOTORS.NS IOC.NS TATAMOTORS.NS     IOC.NS   
Date                                             

In [24]:
prop = final_result["TATAMOTORS.NS"][0]
result = final_result["TATAMOTORS.NS"][1]

print(result[0])
print(prop)

0
[[0.7047337  0.29526627]]


In [ ]:
loaded_model = {}
with open('../TrainedModel/xg/{}_model_2.pkl'.format("IOC.NS"), 'rb') as f:
    loaded_model = pickle.load(f)

In [ ]:
# df_tts = finaldata["IOC.NS"].iloc[:, : -1]
df_tts
train_yhat = loaded_model.predict(finaldata["IOC.NS"])
train_yhat_proba = loaded_model.predict_proba(finaldata["IOC.NS"])

In [ ]:
print(train_yhat,train_yhat_proba)